# EE512-project    
## Class Project1   Options Pricing via Monte Carlo Estimation
### Huihan Gao 4106-6450-95

In [139]:
import numpy as np
from scipy import stats

# (a) payoff for the call option

The function *payoff* takes a GBM path and a strike price K as input.The output is the payoff for the call option *max(ST-K,0)*.

In [140]:
def payoff(path, K):
    return max(path[-1] - K, 0)

# (b)expected value of European call options

The function *generate_GBM* generates a GBM path following:<br/>
St+1 = St + mu * St * dt + sigma * St * rand * sqrt(dt)<br/>
rand~N(0,1)<br/>
The input is initial value *S0*, length *T*, parameters *mu* and *sigma*.<br/>
Assume *dt = 1/365*.

In [141]:
def generate_GBM(S0,T,mu,sigma):  
    path = np.zeros(T)  #a GBM path with length T 
    path[0] = S0  
    dt = 1/365 
    rand = np.random.randn(T)  # T random values~N(0,1)
    for i in range(1,T):
        path[i] = path[i-1] + mu * path[i-1] * dt  + sigma * path[i-1] * rand[i] * np.sqrt(dt)  
    return path
#generate_GBM(100,500,0.05,0.2)

Set 5 sets of GBM parameters as (0.03,0.2),(0.05,0.2),(0.05,0.3),(0.1,0.3),(0.1,0.4).Assume the strike price of the European call option is 95.<br/>
Assumptions: constant asset volatilities, no discounting.<br/>
For each set of parameters, generate 10000 GBM paths and calcalulate the payoff for the call option. The estimation of the expected value of the European option is mean of the 10000 payoffs. 

In [143]:
S0 = 100 
K = 95
T = 500    
N = 10000   
mus = [0.03,0.05,0.05,0.1,0.1]  
sigmas = [0.2,0.2,0.3,0.3,0.4]
value = np.zeros((5,N))    #store the payoffs     
path = np.zeros((5,N,T))   #store the GBM paths generated and is used in (d)
for j in range(5):
    for i in range(N):
        G1 = generate_GBM(S0,T,mus[j],sigmas[j])  #generate a GBM path
        path[j,i] = G1
        value[j,i] = payoff(path[j,i], K) #calculate the payoff
exp_value = np.mean(value,axis=1)  #calculate the mean of payoffs 
exp_value #estimation of expected value of the 5 European options

array([14.23244502, 16.81912269, 20.30729286, 26.30765463, 30.89266284])

# confidence interval

The function *interval* calculates the confidence interval based on observed data.The input is confidence level, mean and standard deviation of observed data and sample size.<br/>
confidence interval = (mean - (Z * std / sqrt(N), mean + (Z * std / sqrt(N))

In [144]:
def interval(confidence, mean, std, N):
    alpha = 1 - confidence
    z_score = scipy.stats.norm.isf(alpha / 2)  #calculate z-score
    me = z_score * std / np.sqrt(N)
    lower = mean - me
    upper = mean + me
    return [lower,upper]

In [145]:
confidence = 0.99  #set confidence level as 99%
std = np.std(value,axis=1)   #calculate standard deviation
intervals = np.zeros((5,2))
for i in range(5):
    intervals[i] = interval(confidence, exp_value[i], std[i], N)
intervals  #confidence intervals

array([[13.73973125, 14.72515878],
       [16.28443654, 17.35380884],
       [19.52671673, 21.087869  ],
       [25.4137231 , 27.20158615],
       [29.64111867, 32.14420701]])

# (c) discounted expected payoff for European call options

Assume the discount is calculated by day. <br/>
*discount rate = (r/365)^T* <br/>
*discounted value = max(ST * discount rate-K, 0)*

In [147]:
r = 0.05
dis_value = np.zeros([5,N])
dis_rate = np.power(1+r/365,500)  #discount rate for T days
for j in range(5):
    for i in range(T):
        dis_value[j,i] = payoff(path[j,i]*dis_rate, K)
dis_exp_value = np.mean(dis_value,axis=1)
dis_exp_value

array([0.96093016, 1.19652698, 1.41451462, 1.75105421, 1.84300443])

# (d) expected value of Asian call options

Use the GBM paths generated in (b). Change the method of calculating payoffs into *max(Avg(S50,S100,...,ST)-K,0)*.The other steps stay the same.

In [148]:
path0 = path[:,:,::-50]    #(ST,S(T-50),...,S50)
S_ave = np.mean(path0,axis=2)  #S_ave = average(S50,S100,...,ST)
value2 = np.zeros([5,N])    #the payoffs
for j in range(5):
    for i in range(N):
        value2[j,i] = payoff([S_ave[j,i]],K)
exp_value2 = np.mean(value2,axis=1)
exp_value2    #estimation of expected value of the 5 *Asian* options

array([ 9.87758439, 11.28647632, 13.35148656, 16.6492591 , 19.40695462])

In [149]:
exp_value  #estimation of expected value of the 5 *European* options

array([14.23244502, 16.81912269, 20.30729286, 26.30765463, 30.89266284])

#### The value of Asian call option is less than the value of European call option with same parameters.

# (e) Jump-Diffusion

The function *generate_GBM_jump* generates a GBM path with jump following:<br/>
St+1 = St + mu * St * dt + sigma * St * rand * sqrt(dt) + M<br/>
rand~N(0,1)<br/> 
M follows a Poisson distribution with the expectation of interval = lamda,which means the jump is expected to happen *lamda* times in a day and jump for 1 in price each time.  <br/>


In [150]:
def generate_GBM_jump(S0,T,mu,sigma,lamda):
    path = np.zeros(T)   
    path[0] = S0  
    dt = 1/365 
    rand = np.random.randn(T)  # T random values~N(0,1)
    M = np.random.poisson(lam=lamda, size=T)   #M follows a Poisson distribution
    for i in range(1,T):
        path[i] = path[i-1] + mu * path[i-1] * dt  + sigma * path[i-1] * rand[i] * np.sqrt(dt) + M[i]
    return path

In [152]:
lamda = 0.01
value3 = np.zeros([5,N]) #values of payoffs     
path2 = np.zeros((5,N,T)) 
for j in range(5):
    for i in range(N):
        G1 = generate_GBM_jump(S0, T, mus[j], sigmas[j], lamda)     #generate a GBM path with jump
        path2[j,i] = G1
        value3[j,i] = payoff(path2[j,i],K)
exp_value3 = np.mean(value3,axis=1)  
exp_value3   #the expected value

array([18.14483658, 20.47520179, 24.49989224, 31.08295396, 35.17724122])